<a href="https://colab.research.google.com/github/MarkStephens060482/oop_examples/blob/main/Prac2Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Truncated Skewed Normal Distribution

In [ ]:
"""
Truncated Skew Normal Random Number Generator
This uses a simpson method approximation for the integral of the pdf of the skew normal distribution
to form the cumulative distribution function.
Attributes:
    m - mean of distribution.
    s - standard deviation of distribution
    l - skew of distribution
    a - minimum value of truncated distribution
    b - maximum value of truncated distriubtion
    h - step size of numerical integration
"""
import numpy as np
from scipy import special

def znormpdf(x):
    return ((2*np.pi)**(-0.5))*np.exp(-0.5*(x**2))


def znormcdf(x):
    return 0.5*(1 + special.erf(x/np.sqrt(2)))

def skewnormpdf(x,m,s,l):
    return (2/s)*znormpdf((x-m)/s)*znormcdf(l*(x-m)/s)

def simpson_sncdf(x,m,s,l,h):
    # calculating the number of steps for step size of h to span from m-6*s to x
    n = int((x-(m-6*s))/h)
    array_prob = []
    array_z = []
    # find initial integration value and corresponding z_score
    integration = skewnormpdf(m-6*s,m,s,l)
    z_score = ((m-6*s)-m)/s
    array_prob.append(integration*h/3)
    array_z.append(z_score)
    # find all integration values at an interval of h apart with corresponding z_score.
    for i in range(1,n):
        k = (m-6*s) + i*h
        if i % 2 == 0:
            integration = integration + 2 * skewnormpdf(k,m,s,l)
        else:
            integration = integration + 4 * skewnormpdf(k,m,s,l)
        z_score = (k-m)/s
        array_prob.append(integration*h/3)
        array_z.append(z_score)
    # Finding final integration value and corresponding z_score
    integration = integration + skewnormpdf(x,m,s,l)
    z_score = (x-m)/s
    array_prob.append(integration*h/3)
    array_z.append(z_score)
    return array_z, array_prob

def approx_inverse_sncdf(m,s,l,a,b,h):
    max_prob = simpson_sncdf(b,m,s,l,h)[1][-1]
    min_prob = simpson_sncdf(a,m,s,l,h)[1][-1]
    val = min_prob + np.random.uniform()*(max_prob-min_prob)
    arr = simpson_sncdf(1000,m,s,l,h)
    index = np.abs(arr[1] - val).argmin()
    return arr[0][index]

def trucskewnormrand(m,s,l,a,b,h):
    rand_num = m + s*approx_inverse_sncdf(m,s,l,a,b,h)
    return rand_num